## 1.加载keras模块

In [22]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D,ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input, VGG16
from keras import backend as K
import numpy as np


### 加载预训练权值



### 定义新加入的layers

In [27]:
img_width, img_height = 150, 150
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
# 定义网络输入层shape
input_tensor=Input(shape=input_shape)
#声明VGG网络，并指定加载imagenet预训练模型
model = VGG16(weights="imagenet",include_top=False,input_tensor=input_tensor)
#向VGG网络追加一些全连接层和输出层
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
#指定输出层为2个神经元做2分类，激活函数选用softmax
top_model.add(Dense(2, activation='softmax'))
#把上述网络每一层layer拷贝至一个新的网络，形成一个整体网络
new_model = Sequential()
for layer in model.layers:
    new_model.add(l)
new_model.add(top_model)


### 设置不需微调的layers的trainable属性
并调用compile函数重新编译网络

In [33]:
from keras.optimizers import SGD
#在新的整体网络上设置对所有原VGG层均不参与微调
for layer in new_model.layers[:]:
    layer.trainalbe=False
new_model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

#compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
#注意此处应选用何种loss function

### 定义ImageDataGenerator


In [34]:
train_data_dir = r'/home/zhou/zhou/实验10/dogs-vs-cats/train'
validation_data_dir = r'/home/zhou/zhou/实验10/dogs-vs-cats/validation'
nb_train_samples = 10835
nb_validation_samples = 4000
epochs = 1
batch_size = 20


# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)
#此处调用flow_from_directory函数生成train_generator和validation_generator,注意设置class_mode

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')




Found 10835 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


### 训练模型



In [35]:
new_model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/1
541/541 [==============================] - 7715s 14s/step - loss: 0.2909 - accuracy: 0.8627 - val_loss: 0.0325 - val_accuracy: 0.9457


### 使用训练后模型预测图像





In [38]:
import cv2
img = cv2.resize(cv2.imread(r'/home/zhou/zhou/实验10/dogs-vs-cats/test/1.jpg'), (img_width, img_height)).astype(np.float32)
# img[:,:,0] -= 103.939
# img[:,:,1] -= 116.779
# img[:,:,2] -= 123.68
#img = img.transpose((2,0,1))
x = img_to_array(img)

x = np.expand_dims(x, axis=0)

#x = preprocess_input(x)

score = new_model.predict(x)


print(score)

ModuleNotFoundError: No module named 'cv2'

###此处尝试只锁定VGG头四层

In [ ]:
#在新的整体网络上设置原VGG前四层不参与微调
for layer in new_model.layers[:4]:
    .train

#编译网络后进行训练



In [ ]:
#使用训练后模型预测图像

